In [8]:
import numpy as np
import pandas as pd

In [9]:
loan = pd.read_csv('lending-club-data.csv')

C:\SowmyaSoftwares\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
loan['safe_loans'] = loan['bad_loans'].apply(lambda x:+1 if x==0 else -1)

In [11]:
loan.drop('bad_loans',1,inplace=True)

In [12]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

In [13]:
loan_data = loan[features + [target]]

In [14]:
import json

In [15]:
tr = open('module-5-assignment-2-train-idx.json','r')
train_ID = json.loads(tr.read())

In [16]:
te = open('module-5-assignment-2-test-idx.json','r')
test_ID = json.loads(te.read())

In [17]:
train_data = loan_data.iloc[train_ID]
test_data = loan_data.iloc[test_ID]

In [19]:
train_data.head(1)

,grade,term,home_ownership,emp_length,safe_loans
1,C,60 months,RENT,< 1 year,-1


In [23]:
train_f = pd.get_dummies(train_data,features)
test_f = pd.get_dummies(test_data,features)

In [22]:
train_f.shape

(37224, 26)

In [24]:
test_f.shape

(9284, 26)

In [38]:
def intermediate_node_num_mistakes(labels):
    
    #label_array = np.array(labels)
    safe_loans = np.count_nonzero(labels == +1)
    risky_loans = np.count_nonzero(labels == -1)
    mistakes = min(safe_loans,risky_loans)
    return mistakes

In [67]:
def best_splitting_feature(df,feature_list,label_name):
    best_feature = None
    best_error= len(df)
    for feat in feature_list:
        left_split = df[df[feat]==0][label_name]
        left_split_mistakes = intermediate_node_num_mistakes(left_split)
        right_split = df[df[feat]==1][label_name]
        right_split_mistakes = intermediate_node_num_mistakes(right_split)
        class_error = (left_split_mistakes+ right_split_mistakes)/len(df)
        
        if class_error < best_error:
            best_error = class_error
            best_feature = feat
    return best_feature

In [160]:
best_splitting_feature(train_f,feature_list,'safe_loans')

'term_ 36 months'

In [161]:
def create_leaf(target_values):    
    # Create a leaf node
    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True   }   ## YOUR CODE HERE 
   
    # Count the number of data points that are +1 and -1 in this node.
    num_ones = len(target_values[target_values == +1])
    num_minus_ones = len(target_values[target_values == -1])    

    # For the leaf node, set the prediction to be the majority class.
    # Store the predicted class (1 or -1) in leaf['prediction']
    if num_ones > num_minus_ones:
        leaf['prediction'] = +1      ## YOUR CODE HERE
    else:
        leaf['prediction'] = -1       ## YOUR CODE HERE        

    # Return the leaf node
    return leaf 

In [162]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    
    remaining_features = features
    #print(remaining_features)
    target_values = data[target]
    
    if intermediate_node_num_mistakes(target_values) == 0:
        
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)       
                
    if remaining_features == []:
        
        print("stopping condition 2 reached")
        return create_leaf(target_values)        
    
    if current_depth == max_depth:
        
        print("stopping condition 3 reached")
        return create_leaf(target_values)
            
    splitting_feature = best_splitting_feature(data,features,target)
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)
    print("split on feature %s .(%s,%s)"%(splitting_feature,len(left_split),len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print("Creating leaf node_1st if.")
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print("Creating leaf node_2nd if")
        return create_leaf(right_split[target])
    print('before left tree' +str(current_depth))
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)
    print(left_tree)
    print('inbetween left and right tree' +str(current_depth))
    right_tree = decision_tree_create(right_split,remaining_features,target,current_depth+1 ,max_depth)
    print(right_tree)
    print('after right tree' +str(current_depth))
    
    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}  

In [163]:
feature_list = list(train_f.drop('safe_loans',1).columns)
target

'safe_loans'

In [164]:
my_decision_tree= decision_tree_create(train_f,feature_list,target,current_depth=0,max_depth=6)

split on feature term_ 36 months .(9223,28001)
before left tree0
split on feature grade_A .(9122,101)
before left tree1
split on feature grade_B .(8074,1048)
before left tree2
split on feature grade_C .(5884,2190)
before left tree3
split on feature grade_D .(3826,2058)
before left tree4
split on feature grade_E .(1693,2133)
before left tree5
stopping condition 3 reached
{'splitting_feature': None, 'left': None, 'right': None, 'is_leaf': True, 'prediction': -1}
inbetween left and right tree5
stopping condition 3 reached
{'splitting_feature': None, 'left': None, 'right': None, 'is_leaf': True, 'prediction': -1}
after right tree5
{'is_leaf': False, 'prediction': None, 'splitting_feature': 'grade_E', 'left': {'splitting_feature': None, 'left': None, 'right': None, 'is_leaf': True, 'prediction': -1}, 'right': {'splitting_feature': None, 'left': None, 'right': None, 'is_leaf': True, 'prediction': -1}}
inbetween left and right tree4
split on feature grade_F .(2058,0)
Creating leaf node_1st if

In [165]:
my_decision_tree['splitting_feature']

'term_ 36 months'

In [166]:
def classify(tree, x, annotate = False):
       # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate:
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    else:
        
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        
        if annotate:
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)

In [185]:
print(test_f.iloc[0]['safe_loans'])
print('Predicted class: %s ' % classify(my_decision_tree, test_f.iloc[0],annotate=True
                                       ))

-1
Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1
Predicted class: -1 


In [210]:
def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row in your data
    data['prediction'] = [classify(tree, x) for x in data.to_dict(orient='records')]
    
    accuracy  = np.count_nonzero(data['prediction'] == data['safe_loans'])/len(data)
    return accuracy

In [211]:
evaluate_classification_eror(my_decision_tree, test_f)

0.5771219302024989

In [218]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    split_feature, split_value = split_name.split('_')
    print('                       %s' % name)
    print ('         |---------------|----------------|')
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('  [{0} == 0]               [{0} == 1]    '.format(split_name))
    print ('         |                                |')
    print ('         |                                |')
    print ('         |                                |')
    print ('    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')))

In [219]:
print_stump(my_decision_tree, name='root')

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [220]:
print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])


                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [221]:
print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])

                       grade_A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_B == 0]               [grade_B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [222]:
print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])

ValueError: too many values to unpack (expected 2)

In [223]:
print_stump(my_decision_tree['right']['right'], my_decision_tree['right']['splitting_feature'])

(leaf, label: 1)
